# Introduction/Business Model

The problem I will be solving with the Foursquare location database will be to help a firm confirm if the optimal location to open a hotel in Vancouver, BC is off the street from Canada Place, in Downtown Vancouver. The target audience are the shareholders of the firm trying to open the hotel. They required my services to confirm if they have selected a good place to open the hotel.

I will need to use the Foursquare Api to find main tourist destinations like stadiums, beaches, trails, hockey arenas, parks, art galleries, plazas, waterfronts, sculpture gradens, outdoor sculptures and scenic lookouts within 15km of the proposed site of the hotel.

# Data

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import requests

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         395 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5

Let us get our data, below we search for venues close to Canada Place

In [2]:
radius = 15000

CLIENT_ID = 'U2GYDX22HCMBF2RGMP5BZ1ATYLX0QJU1LC3LCNU1LJYBENTD' # your Foursquare ID
CLIENT_SECRET = 'NVY5WHASZCPPPOJDSYKAWUSG3OSOLGWXS4S12WY5K1P2RNT1' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 101

latitude = 49.288635
longitude = -123.11112

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=U2GYDX22HCMBF2RGMP5BZ1ATYLX0QJU1LC3LCNU1LJYBENTD&client_secret=NVY5WHASZCPPPOJDSYKAWUSG3OSOLGWXS4S12WY5K1P2RNT1&ll=49.288635,-123.11112&v=20180604&radius=15000&limit=101'

In [3]:
results = requests.get(url).json()
'There are {} around Canada Place.'.format(len(results['response']['groups'][0]['items']))

'There are 100 around Canada Place.'

In [4]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4bbcbcf88ec3d13ab0261b28',
  'name': 'Fairmont Pacific Rim',
  'location': {'address': '1038 Canada Pl.',
   'crossStreet': 'btwn Burrard & Thurlow',
   'lat': 49.28822730314015,
   'lng': -123.11693176354468,
   'labeledLatLngs': [{'label': 'display',
     'lat': 49.28822730314015,
     'lng': -123.11693176354468}],
   'distance': 424,
   'postalCode': 'V6C 0B9',
   'cc': 'CA',
   'city': 'Vancouver',
   'state': 'BC',
   'country': 'Canada',
   'formattedAddress': ['1038 Canada Pl. (btwn Burrard & Thurlow)',
    'Vancouver BC V6C 0B9',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d1fa931735',
    'name': 'Hotel',
    'pluralName': 'Hotels',
    'shortName': 'Hotel',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 

In [5]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [6]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(55)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Fairmont Pacific Rim,Hotel,1038 Canada Pl.,btwn Burrard & Thurlow,49.288227,-123.116932,"[{'label': 'display', 'lat': 49.28822730314015...",424,V6C 0B9,CA,Vancouver,BC,Canada,"[1038 Canada Pl. (btwn Burrard & Thurlow), Van...",NaN,4bbcbcf88ec3d13ab0261b28
1,Revolver,Coffee Shop,325 Cambie St,btwn Hastings & Cordova,49.283187,-123.109288,"[{'label': 'display', 'lat': 49.2831867506594,...",620,V6B 1H7,CA,Vancouver,BC,Canada,"[325 Cambie St (btwn Hastings & Cordova), Vanc...",Gastown,4d9b41359298b1f7635b5338
2,Meat & Bread,Sandwich Place,370 Cambie St,btwn Cordova & Hastings,49.282646,-123.109499,"[{'label': 'display', 'lat': 49.2826459299748,...",677,V6B 2N3,CA,Vancouver,BC,Canada,"[370 Cambie St (btwn Cordova & Hastings), Vanc...",NaN,4cc72e8b01fb236a8d70bbba
3,Rosewood Hotel Georgia,Hotel,801 W Georgia St,at Howe St,49.283429,-123.118911,"[{'label': 'display', 'lat': 49.28342914255638...",809,V6C 1P7,CA,Vancouver,BC,Canada,"[801 W Georgia St (at Howe St), Vancouver BC V...",NaN,4d5ec8ce29ef236ae0cb9059
4,Coal Harbour Seawall,Trail,W Cordova St.,NaN,49.291304,-123.123276,"[{'label': 'display', 'lat': 49.291304, 'lng':...",931,V7J 2E1,CA,Vancouver,BC,Canada,"[W Cordova St., Vancouver BC V7J 2E1, Canada]",NaN,4ab03489f964a520706620e3
5,Gotham Steakhouse & Cocktail Bar,Steakhouse,615 Seymour St.,btwn Dunsmuir & W Georgia,49.282830,-123.115865,"[{'label': 'display', 'lat': 49.28283023000359...",732,V6B 3K3,CA,Vancouver,BC,Canada,"[615 Seymour St. (btwn Dunsmuir & W Georgia), ...",NaN,4aa7f22bf964a520214e20e3
6,iLoveKickboxing.com Vancouver,Boxing Gym,136 Powell Street,Gastown,49.283143,-123.100976,"[{'label': 'display', 'lat': 49.283143, 'lng':...",957,V6A 1G1,CA,Vancouver,BC,Canada,"[136 Powell Street (Gastown), Vancouver BC V6A...",NaN,5660efe3498e64f8e2970c9e
7,Jack Poole Plaza,Plaza,Vancouver Convention Centre,at Thurlow St,49.289656,-123.117621,"[{'label': 'display', 'lat': 49.28965628946816...",485,V6C 3E1,CA,Vancouver,BC,Canada,"[Vancouver Convention Centre (at Thurlow St), ...",NaN,4c083508009a0f479fc2e5bf
8,Birds & The Beets,Café,55 Powell St,NaN,49.283556,-123.102940,"[{'label': 'display', 'lat': 49.28355627550181...",820,V6A 1E9,CA,Vancouver,BC,Canada,"[55 Powell St, Vancouver BC V6A 1E9, Canada]",NaN,553ac7e1498e04e927f8762d
9,Loden Hotel,Hotel,1177 Melville St.,at Butte St.,49.287690,-123.123574,"[{'label': 'display', 'lat': 49.28769003824368...",910,V6E 0A3,CA,Vancouver,BC,Canada,"[1177 Melville St. (at Butte St.), Vancouver B...",NaN,4b18a7acf964a52002d523e3


This is the data I will be working with.

It consists of many different columns, however, I will only be working with a couple specific features. I will use the name feature to name the venue. I will use the categories to take only some specific rows that include main tourist destinations like stadiums, beaches, trails, hockey arenas, parks, art galleries, plazas, waterfronts, sculpture gardens, outdoor sculptures and scenic lookouts. The Latitude and Longitude will be taken so I will be able to plot the locations on a map.

If there are at least 20 of the destinations specified above in the dataset, then I shall be able to come to the conclusion that the location for the hotel is optimal and we can assure that there are enough venues around the area to attract tourists and keep them busy.

# Methodology

What I did with the data was that I filtered it to include some specific columns. Once
that was done, used dataset.index to find the number of rows in the dataset. Each row
represents one tourist destination close to the proposed site of the hotel. I also created a
folium map to visualize the different destinations.

# Solving the problem

Lets make a dataset of only some specific rows in which the categories column includes main tourist destinations like stadiums, beaches, trails, hockey arenas, parks, art galleries, plazas, waterfronts, sculpture gardens, outdoor sculptures and scenic lookouts.

In [7]:
df = []
num = 0
for row in dataframe_filtered.itertuples():
    
    if dataframe_filtered['categories'][num] == 'Trail':
        df.append(row)
    if dataframe_filtered['categories'][num] == 'Hockey Arena':
        df.append(row)
    if dataframe_filtered['categories'][num] == 'Park':
        df.append(row)
    if dataframe_filtered['categories'][num] == 'Art Gallery':
        df.append(row)
    if dataframe_filtered['categories'][num] == 'Plaza':
        df.append(row)
    if dataframe_filtered['categories'][num] == 'Waterfront':
        df.append(row)
    if dataframe_filtered['categories'][num] == 'Sculpture Garden':
        df.append(row)
    if dataframe_filtered['categories'][num] == 'Outdoor Sculpture':
        df.append(row)
    if dataframe_filtered['categories'][num] == 'Plaza':
        df.append(row)
    if dataframe_filtered['categories'][num] == 'Waterfront':
        df.append(row)
    if dataframe_filtered['categories'][num] == 'Outdoor Sculpture':
        df.append(row)
    if dataframe_filtered['categories'][num] == 'Scenic Lookout':
        df.append(row)
    
    num += 1    

In [8]:
dataset = pd.DataFrame(df)

In [9]:
dataset

,Index,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,4,Coal Harbour Seawall,Trail,W Cordova St.,NaN,49.291304,-123.123276,"[{'label': 'display', 'lat': 49.291304, 'lng':...",931,V7J 2E1,CA,Vancouver,BC,Canada,"[W Cordova St., Vancouver BC V7J 2E1, Canada]",NaN,4ab03489f964a520706620e3
1,7,Jack Poole Plaza,Plaza,Vancouver Convention Centre,at Thurlow St,49.289656,-123.117621,"[{'label': 'display', 'lat': 49.28965628946816...",485,V6C 3E1,CA,Vancouver,BC,Canada,"[Vancouver Convention Centre (at Thurlow St), ...",NaN,4c083508009a0f479fc2e5bf
2,7,Jack Poole Plaza,Plaza,Vancouver Convention Centre,at Thurlow St,49.289656,-123.117621,"[{'label': 'display', 'lat': 49.28965628946816...",485,V6C 3E1,CA,Vancouver,BC,Canada,"[Vancouver Convention Centre (at Thurlow St), ...",NaN,4c083508009a0f479fc2e5bf
3,12,Coal Harbour Park,Park,W Hastings St,Jervis St,49.289936,-123.125002,"[{'label': 'display', 'lat': 49.28993619768085...",1018,NaN,CA,Vancouver,BC,Canada,"[W Hastings St (Jervis St), Vancouver BC, Canada]",NaN,4c19649998f4a5932b0f00f6
4,14,Harbour Green Park,Park,1200 West Cordova St.,Bute St.,49.290062,-123.121719,"[{'label': 'display', 'lat': 49.29006215585809...",785,V6C 3R4,CA,Vancouver,BC,Canada,"[1200 West Cordova St. (Bute St.), Vancouver B...",NaN,4c474fc476d72d7fa3c03c4d
5,19,Rogers Arena,Hockey Arena,800 Griffiths Way,at Abbott St,49.277923,-123.108723,"[{'label': 'display', 'lat': 49.27792296883324...",1205,V6B 6G1,CA,Vancouver,BC,Canada,"[800 Griffiths Way (at Abbott St), Vancouver B...",NaN,4aa6f23df964a520954b20e3
6,22,Seaside Seawall,Trail,Quebec St,NaN,49.272541,-123.104061,"[{'label': 'display', 'lat': 49.27254136682504...",1863,NaN,CA,Vancouver,BC,Canada,"[Quebec St, Vancouver BC, Canada]",NaN,4df29cb0fa76abc3d86a240a
7,27,Stanley Park Harbourfront Seawall,Waterfront,1999 Georgia St,Stanley Park Causeway Rd,49.297193,-123.130671,"[{'label': 'display', 'lat': 49.29719336520892...",1709,NaN,CA,Vancouver,BC,Canada,"[1999 Georgia St (Stanley Park Causeway Rd), V...",NaN,4aa72e38f964a520274c20e3
8,27,Stanley Park Harbourfront Seawall,Waterfront,1999 Georgia St,Stanley Park Causeway Rd,49.297193,-123.130671,"[{'label': 'display', 'lat': 49.29719336520892...",1709,NaN,CA,Vancouver,BC,Canada,"[1999 Georgia St (Stanley Park Causeway Rd), V...",NaN,4aa72e38f964a520274c20e3
9,32,Totem Poles in Stanley Park,Sculpture Garden,Stanley Park,NaN,49.299146,-123.121155,"[{'label': 'display', 'lat': 49.2991460501812,...",1378,NaN,CA,Vancouver,BC,Canada,"[Stanley Park, Vancouver BC, Canada]",Stanley Park,4b58e9ecf964a5208d7228e3


Now that we have all the rows in which the categories are the ones we need, we can get rid of some columns we don't need to solve our problem and just clutter the dataset. 

Lets remove the columns Index, address, crossStreet, labeledLatLngs, postalCode, cc, city, state, country, formattedAddress, neighborhood, distance and id from our dataset

In [10]:
del dataset['Index'] 
del dataset['address']
del dataset['crossStreet']
del dataset['labeledLatLngs']
del dataset['postalCode']
del dataset['cc']
del dataset['city']
del dataset['state']
del dataset['country']
del dataset['formattedAddress']
del dataset['neighborhood']
del dataset['distance']
del dataset['id']

In [11]:
dataset

,name,categories,lat,lng
0,Coal Harbour Seawall,Trail,49.291304,-123.123276
1,Jack Poole Plaza,Plaza,49.289656,-123.117621
2,Jack Poole Plaza,Plaza,49.289656,-123.117621
3,Coal Harbour Park,Park,49.289936,-123.125002
4,Harbour Green Park,Park,49.290062,-123.121719
5,Rogers Arena,Hockey Arena,49.277923,-123.108723
6,Seaside Seawall,Trail,49.272541,-123.104061
7,Stanley Park Harbourfront Seawall,Waterfront,49.297193,-123.130671
8,Stanley Park Harbourfront Seawall,Waterfront,49.297193,-123.130671
9,Totem Poles in Stanley Park,Sculpture Garden,49.299146,-123.121155


Using dataset.index, we can see the if we start from 0 and step by 1, the stop point of the indices is 25, so we have 25 major tourist destinations nearby

In [12]:
dataset.index

RangeIndex(start=0, stop=25, step=1)

We have over 20 major tourist destinations nearby, so that tells us that near Canada Place is the ideal area to open a hotel in Vancouver

In [13]:
!pip install folium
import folium

In [15]:
latitude = 49.288219
longitude = -123.115723

venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataset.lat, dataset.lng, dataset.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

# Results and discussion

After using dataset.index, I found that there were 25 rows in our dataset. Each row
represented a major tourist destination. So there were 25 different tourist destinations close
to the proposed site of our hotel.

I was hired by a Vancouver based firm to confirm if near Canada Place was the
optimal location to open a hotel. I set a base number of at least 20 main tourist destinations
like stadiums, beaches, trails, hockey arenas, parks, art galleries, plazas, waterfronts,
sculpture gardens, outdoor sculptures and scenic lookouts. If there were over 20 main tourist
destinations, then I could confirm if the location was a good place to open a hotel. So I then
used the Foursquare API to query for venues nearby. I then filtered the dataset to only
include the main tourist destinations I specified above so I could get an idea for how many
destinations were there. I found that there were 25 of such destinations within 15km of the
proposed site of the hotel. I could now recommend the proposed site for a hotel.